In [1]:
using Pkg
for p in ["Knet", "Plots", "IterTools","PyCall"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
using DelimitedFiles
using Knet: KnetArray, accuracy, progress, minibatch, cycle, adam, xavier_uniform
using Plots
using IterTools: ncycle, takenth, take
using Base.Iterators: flatten
using LinearAlgebra

In [2]:
include("utils.jl")
include("models.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [3]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("nell")

InterruptException: InterruptException:

In [ ]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end

arguments = args(200, 0.01, 1e-5, 64, 0.1)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            adj,
            arguments.pdrop)

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w)) 
#(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train])

In [ ]:
output = model(features)

In [ ]:
labels_decoded = mapslices(argmax, labels ,dims=2)[:]

In [ ]:
nll(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
using AutoGrad
J = @diff model(features, labels_decoded)
grad(J, params(model)[1])

In [ ]:
dtrn =  minibatch(features, labels_decoded[:], length(labels_decoded))

In [ ]:
function train_with_results(model, dtrn, epoch, lr)
    training = adam(model, ncycle(dtrn, epoch), lr=lr)
    snapshot() = model(dtrn)
    snapshots = (snapshot() for x in takenth(progress(training),length(dtrn)))
    res = collect(flatten(snapshots))
    return res
end

In [ ]:
results = train_with_results(model, dtrn, arguments.epochs, arguments.lr)
trnloss = Array{Float32}(results)
plot(trnloss, ylim=(0.0,2.0),labels=[:trnloss],xlabel="Epochs",ylabel="Loss")

In [ ]:
output = model(features)
accuracy(output[:,idx_train], labels_decoded[idx_train])

In [ ]:
output = model(features)
accuracy(output[:,idx_test], labels_decoded[idx_test])